In [ ]:
!pip install ../input/detectron-05/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links ../input/detectron-05/whls 
!pip install ../input/detectron-05/whls/detectron2-0.5/detectron2 --no-index --find-links ../input/detectron-05/whls 

In [ ]:
!cp ../input/resnest200/mask_rcnn_ResNeSt200.yaml /opt/conda/lib/python3.7/site-packages/detectron2/model_zoo/configs/COCO-InstanceSegmentation/
!cp ../input/resnest200/resnet.py /opt/conda/lib/python3.7/site-packages/detectron2/modeling/backbone/
!cp ../input/resnest200/splat.py /opt/conda/lib/python3.7/site-packages/detectron2/modeling/backbone/

In [ ]:
ls

In [ ]:
import detectron2
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg, CfgNode
from PIL import Image
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from fastcore.all import *
detectron2.__version__
import torch

import pycocotools.mask as mask_util
from tqdm.notebook import tqdm

In [ ]:
dataDir=Path('../input/sartorius-cell-instance-segmentation')

In [ ]:
# make detectron2 return per pixel mask score
def paste_masks_in_image(masks, boxes, image_shape, threshold=0.5):
    """
    Copy pasted from detectron2.layers.mask_ops.paste_masks_in_image and deleted thresholding of the mask
    """
#     print(torch.unique(masks))
    assert masks.shape[-1] == masks.shape[-2], "Only square mask predictions are supported"
    N = len(masks)
    if N == 0:
        return masks.new_empty((0,) + image_shape, dtype=torch.uint8)
    if not isinstance(boxes, torch.Tensor):
        boxes = boxes.tensor
    device = boxes.device
    assert len(boxes) == N, boxes.shape

    img_h, img_w = image_shape

    # The actual implementation split the input into chunks,
    # and paste them chunk by chunk.
    if device.type == "cpu":
        # CPU is most efficient when they are pasted one by one with skip_empty=True
        # so that it performs minimal number of operations.
        num_chunks = N
    else:
        # GPU benefits from parallelism for larger chunks, but may have memory issue
        num_chunks = int(np.ceil(N * img_h * img_w * BYTES_PER_FLOAT / GPU_MEM_LIMIT))
        assert (
            num_chunks <= N
        ), "Default GPU_MEM_LIMIT in mask_ops.py is too small; try increasing it"
    chunks = torch.chunk(torch.arange(N, device=device), num_chunks)

    img_masks = torch.zeros(
        N, img_h, img_w, device=device, dtype=torch.float32
    )
#     print(torch.unique(masks))
    for inds in chunks:
        masks_chunk, spatial_inds = _do_paste_mask(
            masks[inds, None, :, :], boxes[inds], img_h, img_w, skip_empty=device.type == "cpu"
        )
        img_masks[(inds,) + spatial_inds] = masks_chunk
#     print(torch.unique(img_masks))
    return img_masks

from typing import Any, Iterator, List, Union
import numpy as np

def BitMasks__init__(self, tensor: Union[torch.Tensor, np.ndarray]):
    """
    Args:
        tensor: bool Tensor of N,H,W, representing N instances in the image.
    """
    device = tensor.device if isinstance(tensor, torch.Tensor) else torch.device("cpu")
    tensor = torch.as_tensor(tensor, dtype=torch.float32, device=device)
    assert tensor.dim() == 3, tensor.size()
    self.image_size = tensor.shape[1:]
    self.tensor = tensor
    
detectron2.structures.masks.BitMasks.__init__.__code__ = BitMasks__init__.__code__

detectron2.layers.mask_ops.paste_masks_in_image.__code__ = paste_masks_in_image.__code__

In [ ]:
def get_predictor_v21():
    cfg = get_cfg()

    cfg.MODEL.RESNETS.RADIX = 1
    cfg.MODEL.RESNETS.DEEP_STEM = False
    cfg.MODEL.RESNETS.AVD = False
    # Apply avg_down to the downsampling layer for residual path 
    cfg.MODEL.RESNETS.AVG_DOWN = False
    cfg.MODEL.RESNETS.BOTTLENECK_WIDTH = 64

    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_ResNeSt200.yaml"))
    cfg.INPUT.MASK_FORMAT='bitmask'
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
    cfg.MODEL.WEIGHTS = '../input/sartoriusnew-mask-rcnn-v21/model_0017999.pth'
    # cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.99 # keep almost every instances

    cfg.TEST.DETECTIONS_PER_IMAGE = 10000

    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024   
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  

    cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 1024
    cfg.MODEL.RPN.POSITIVE_FRACTION = 0.7

    # img size
    cfg.INPUT.MIN_SIZE_TRAIN = (1024, )
    cfg.INPUT.MAX_SIZE_TRAIN = (2000, )
    cfg.INPUT.MIN_SIZE_TEST = 1024
    cfg.INPUT.MAX_SIZE_TEST = 2000

    cfg.TEST.AUG.MIN_SIZES = (1024, )
    cfg.TEST.AUG.FLIP = False
    
    predictor = DefaultPredictor(cfg)
    
    return cfg, predictor

def get_predictor_v16():
    cfg = get_cfg()

    cfg.MODEL.RESNETS.RADIX = 1
    cfg.MODEL.RESNETS.DEEP_STEM = False
    cfg.MODEL.RESNETS.AVD = False
    # Apply avg_down to the downsampling layer for residual path 
    cfg.MODEL.RESNETS.AVG_DOWN = False
    cfg.MODEL.RESNETS.BOTTLENECK_WIDTH = 64

    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_ResNeSt200.yaml"))
    cfg.INPUT.MASK_FORMAT='bitmask'
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3 
    cfg.MODEL.WEIGHTS = '../input/sartoriusnew-mask-rcnn-v16/model_0028999.pth'
    # cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.99 # keep almost every instances

    cfg.TEST.DETECTIONS_PER_IMAGE = 10000

    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 1024   
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3  

    cfg.MODEL.RPN.BATCH_SIZE_PER_IMAGE = 1024
    cfg.MODEL.RPN.POSITIVE_FRACTION = 0.7

    # img size
    cfg.INPUT.MIN_SIZE_TRAIN = (1024, )
    cfg.INPUT.MAX_SIZE_TRAIN = (2000, )
    cfg.INPUT.MIN_SIZE_TEST = 1024
    cfg.INPUT.MAX_SIZE_TEST = 2000

    cfg.TEST.AUG.MIN_SIZES = (1024, )
    cfg.TEST.AUG.FLIP = False
    
    predictor = DefaultPredictor(cfg)
    
    return cfg, predictor

In [ ]:
# From https://www.kaggle.com/stainsby/fast-tested-rle
def rle_decode(mask_rle, shape=(520, 704)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)  # Needed to align to RLE direction

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def check_overlap(msk):
    msk = msk.astype(np.bool).astype(np.uint8)
    return np.any(np.sum(msk, axis=-1)>1)

def filter_masks_by_area(mask_min_area, outputs):
    pred_class = torch.mode(outputs['instances'].pred_classes)[0]
    area_min = mask_min_area[pred_class]
    take = (outputs['instances'].pred_masks >= 0.5).sum(axis=(1,2)) >= area_min
    outputs['instances'] = outputs['instances'][take]
    return outputs

def get_masks(pred, shape=(520,704)):
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    
#     if(pred_class != 1):
#         return ['']
    
    # filter masks by area min
    pred = filter_masks_by_area([70, 120, 50], pred)
    
    pred_masks = pred['instances'].pred_masks
    pred_masks = pred_masks.cpu().numpy()
    res = []
    used = np.zeros(shape, dtype=int) 
    for mask in pred_masks:
        mask = mask * (1-used)
        used += mask
        res.append(rle_encode(mask))
    return res

In [ ]:
from sklearn.neighbors import KDTree

def calculate_max_IOU_with_gt(targ, pred):
    pred_masks = pred['instances'].pred_masks >= 0.5
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    enc_targs = list(map(lambda x:x['segmentation'], targ))
    ious = mask_util.iou(enc_preds, enc_targs, [0]*len(enc_targs))
    
    return ious.max(axis=1)

def print_log(log):
    for k in log.keys():
        print(k, log[k])
        
def get_overlapping_features(pred):
    pred_masks = pred['instances'].pred_masks >= 0.5
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    ious = mask_util.iou(enc_preds, enc_preds, [0]*len(enc_preds))
    return ious.max(axis=1), ious.min(axis=1), ious.mean(axis=1),\
             ious.std(axis=1), (ious > 0).sum(axis=1)

def get_contour_features(pred):
    masks = (pred['instances'].pred_masks.numpy() >= 0.5).astype('uint8')
    
    data_dict = {
        'centroid_x':[],
        'centroid_y':[],
        'num_contours': [],
        'equi_diameter':[],
        'hull_area':[],
        'solidity':[],
        'is_convex':[],
        'perimeter':[],
        'rotation_ang':[],
        'major_axis_length':[],
        'minor_axis_length':[]
    }
    
    for mask in masks:
        contours, _ = cv2.findContours(mask, 1, 2)
        areas = [cv2.contourArea(cnt) for cnt in contours]
        max_ind = np.argmax(areas)
        
        area = areas[max_ind]
        cnt = contours[max_ind]
        
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        
        hull = cv2.convexHull(cnt)
        hull_area = cv2.contourArea(hull)
        solidity = float(area)/hull_area if hull_area > 0 else -1
        
        equi_diameter = np.sqrt(4*area/np.pi)
        is_convex = int(cv2.isContourConvex(cnt))
        perimeter = cv2.arcLength(cnt,True)
        
        try:
            ellipse = cv2.fitEllipse(cnt)
            _,(major_axis_length, minor_axis_length), rotation_ang = ellipse
        except:
            (major_axis_length, minor_axis_length), rotation_ang = (-1,-1),-1
        
        data_dict['centroid_x'].append(cx)
        data_dict['centroid_y'].append(cy)
        data_dict['num_contours'].append(len(contours))
        data_dict['equi_diameter'].append(equi_diameter)
        data_dict['solidity'].append(solidity)
        data_dict['hull_area'].append(hull_area)
        data_dict['is_convex'].append(is_convex)
        data_dict['perimeter'].append(perimeter)
        data_dict['rotation_ang'].append(rotation_ang)
        data_dict['major_axis_length'].append(major_axis_length)
        data_dict['minor_axis_length'].append(minor_axis_length)
        
    return pd.DataFrame(data_dict)

def get_pixel_scores_features(outputs):
    pred_masks = outputs['instances'].pred_masks
    pred_masks_non_zeros = [mask[mask > 0] for mask in pred_masks]
    min_pscores = [mask.min().item() for mask in pred_masks_non_zeros]
    max_pscores = [mask.max().item() for mask in pred_masks_non_zeros]
    median_pscores = [mask.median().item() for mask in pred_masks_non_zeros]
    mean_pscores = [mask.mean().item() for mask in pred_masks_non_zeros]
                    
    q1_pscores =  [mask.quantile(0.25).item() for mask in pred_masks_non_zeros]
    q3_pscores =  [mask.quantile(0.75).item() for mask in pred_masks_non_zeros]
    std_pscores = [mask.std().item() for mask in pred_masks_non_zeros]
    
    ret = {
        'min_pixel_score':min_pscores,
        'max_pixel_score':max_pscores,
        'median_pixel_score':median_pscores,
        'mean_pixel_score':mean_pscores,
        'q1_pixel_score':q1_pscores,
        'q3_pixel_score':q3_pscores,
        'std_pixel_score':std_pscores
    }
    return pd.DataFrame(ret)

def get_image_pixel_features(im, outputs):
    pred_masks = outputs['instances'].pred_masks
    pred_masks_binary = [mask > 0.5 for mask in pred_masks]
    im_masks = [im[mask,0] for mask in pred_masks_binary]
    
    min_pscores = [mask.min().item() for mask in im_masks]
    max_pscores = [mask.max().item() for mask in im_masks]
    median_pscores = [np.median(mask).item() for mask in im_masks]
    mean_pscores = [mask.mean().item() for mask in im_masks]
                    
    q1_pscores =  [np.quantile(mask, 0.25).item() for mask in im_masks]
    q3_pscores =  [np.quantile(mask, 0.75) for mask in im_masks]
    std_pscores = [mask.std() for mask in im_masks]
    
    ret = {
        'im_min_pixel':min_pscores,
        'im_max_pixel':max_pscores,
        'im_median_pixel':median_pscores,
        'im_mean_pixel':mean_pscores,
        'im_q1_pixel':q1_pscores,
        'im_q3_pixel':q3_pscores,
        'im_std_pixel':std_pscores
    }
    return pd.DataFrame(ret)

def get_kdtree_nb_features(single_features):
    cols = ['centroid_x', 'centroid_y']
    X = single_features[cols]
    tree = KDTree(X)  
    
    ret = dict()
    for r in [25, 50, 75, 100, 150, 200]:
        ind, dist = tree.query_radius(X, r=r, return_distance=True, sort_results=True)  
        ind = [i[1:] for i in ind] # exclude neareast neighbor (itself)
        dist = [d[1:] for d in dist] # exclude neareast neighbor (itself)
        
        ret[f'kdtree_nb_r{r}_count'] = [len(ind) for i in ind]
        
        ret[f'kdtree_nb_r{r}_median_dist'] = [np.median(d) if len(d)>0 else -1 for d in dist]
        ret[f'kdtree_nb_r{r}_mean_dist'] = [d.mean() if len(d)>0 else -1 for d in dist]
        ret[f'kdtree_nb_r{r}_std_dist'] = [np.std(d) if len(d)>0 else -1 for d in dist]
        
        ret[f'kdtree_nb_r{r}_median_area'] = [single_features.loc[i, 'mask_area'].median() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_r{r}_mean_area'] = [single_features.loc[i, 'mask_area'].mean() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_r{r}_std_area'] = [single_features.loc[i, 'mask_area'].std() if len(i)>0 else -1 for i in ind]
        
        ret[f'kdtree_nb_r{r}_median_box_score'] = [single_features.loc[i, 'box_score'].median() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_r{r}_mean_box_score'] = [single_features.loc[i, 'box_score'].mean() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_r{r}_std_box_score'] = [single_features.loc[i, 'box_score'].std() if len(i)>0 else -1 for i in ind]
        
    for k in [2,3,5,7]:
        dist, ind = tree.query(X, k=k, return_distance=True)  
        ind = [i[1:] for i in ind] # exclude neareast neighbor (itself)
        dist = [d[1:] for d in dist] # exclude neareast neighbor (itself)
        
        ret[f'kdtree_nb_top{k}_median_dist'] = [np.median(d) if len(d)>0 else -1 for d in dist]
        ret[f'kdtree_nb_top{k}_mean_dist'] = [d.mean() if len(d)>0 else -1  for d in dist]
        ret[f'kdtree_nb_top{k}_std_dist'] = [np.std(d) if len(d)>0 else -1 for d in dist]
        
        ret[f'kdtree_nb_top{k}_median_area'] = [single_features.loc[i, 'mask_area'].median() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_top{k}_mean_area'] = [single_features.loc[i, 'mask_area'].mean() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_top{k}_std_area'] = [single_features.loc[i, 'mask_area'].std() if len(i)>0 else -1 for i in ind]
        
        ret[f'kdtree_nb_top{k}_median_box_score'] = [single_features.loc[i, 'box_score'].median() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_top{k}_mean_box_score'] = [single_features.loc[i, 'box_score'].mean() if len(i)>0 else -1 for i in ind]
        ret[f'kdtree_nb_top{k}_std_box_score'] = [single_features.loc[i, 'box_score'].std() if len(i)>0 else -1 for i in ind]
        
    return pd.DataFrame(ret)    
    
def get_features(im, outputs):
    pred_masks = outputs['instances'].pred_masks
    
    mask_areas = (pred_masks >= 0.5).sum(axis=(1,2))
    pred_boxes = outputs['instances'].pred_boxes.tensor
    widths = pred_boxes[:,2] - pred_boxes[:,0]
    heights = pred_boxes[:,3] - pred_boxes[:,1]
    box_areas = widths * heights
    
    box_scores = outputs['instances'].scores
    instance_count = len(outputs['instances'])
    
    aspect_ratios = widths / heights
    extents = mask_areas / box_areas
    
    neighbor_iou_max, neighbor_iou_min, neighbor_iou_mean, \
        neighbor_iou_std, neighbor_overlap_count = get_overlapping_features(outputs)
    
    contour_features = get_contour_features(outputs)
    pixel_features = get_pixel_scores_features(outputs)
    im_pixel_features = get_image_pixel_features(im, outputs)
    
    ret = pd.DataFrame({
        'box_score':box_scores,
        'mask_area':mask_areas,
        'box_area':box_areas,
        'box_x1':pred_boxes[:,0],
        'box_y1':pred_boxes[:,1],
        'box_x2':pred_boxes[:,2],
        'box_y2':pred_boxes[:,3],
        'width':widths,
        'height':heights,
        'instance_count':instance_count,
        
        'neighbor_iou_max':neighbor_iou_max,
        'neighbor_iou_min':neighbor_iou_min,
        'neighbor_iou_mean':neighbor_iou_mean,
        'neighbor_iou_std':neighbor_iou_std,
        'neighbor_overlap_count':neighbor_overlap_count,
        
        'aspect_ratio':aspect_ratios,
        'extent':extents
    })
    
    ret = pd.concat([ret, contour_features, pixel_features, im_pixel_features], axis=1)
    kdtree_nb_features = get_kdtree_nb_features(ret)
    ret = pd.concat([ret, kdtree_nb_features], axis=1)
    return ret

In [ ]:
import copy
from detectron2.data.detection_utils import read_image
from detectron2.modeling.test_time_augmentation import GeneralizedRCNNWithTTA, DatasetMapperTTA
from detectron2.modeling.roi_heads.fast_rcnn import fast_rcnn_inference_single_image
from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.structures import Boxes, Instances
from fvcore.transforms import HFlipTransform, NoOpTransform

from contextlib import contextmanager
from itertools import count
import itertools

In [ ]:
def custom_nms(pred, nms_thresh=0.5):
    pred_masks = pred['instances'].pred_masks.detach().cpu().numpy()
    scores = pred['instances'].scores.detach().cpu().numpy()
    # print(pred_masks.dtype)
    if(pred_masks.dtype == np.float32):
        pred_masks = (pred_masks >= 0.5 ).astype('bool')
        # print(pred_masks.dtype)

    # calculate IOU
    enc_preds = [mask_util.encode(np.asarray(p, order='F')) for p in pred_masks]
    ious = mask_util.iou(enc_preds, enc_preds, [0]*len(enc_preds))

    orders = np.arange(len(scores))
    keeps = []

    while len(orders) > 0:

        # append the idx of the instance X with highest score to keeps list
        keeps.append(orders[0])
        # remove the idx of the above instance from the remaining list
        orders = orders[1:]

        # check IOU of the instance X with all the instance in the remaining list
        look_up = ious[keeps[-1], orders]

        # filter those having IOU > nms_thresh
        mask = look_up < nms_thresh
        orders = orders[mask]

        if(len(orders)==0):
            break
    
    new_pred = pred.copy()
    new_pred['instances'] = new_pred['instances'][keeps]
    return new_pred

In [ ]:
@contextmanager
def turn_off_roi_heads(model, attrs):
        """
        Open a context where some heads in `model.roi_heads` are temporarily turned off.
        Args:
            attr (list[str]): the attribute in `model.roi_heads` which can be used
                to turn off a specific head, e.g., "mask_on", "keypoint_on".
        """
        roi_heads = model.roi_heads
        old = {}
        for attr in attrs:
            try:
                old[attr] = getattr(roi_heads, attr)
            except AttributeError:
                # The head may not be implemented in certain ROIHeads
                pass

        if len(old.keys()) == 0:
            yield
        else:
            for attr in old.keys():
                setattr(roi_heads, attr, False)
            yield
            for attr in old.keys():
                setattr(roi_heads, attr, old[attr])

def model_batch_forward(model, inputs, instances=[None]):
    outputs = model.inference(inputs, 
                   instances if instances[0] is not None else None,
                   do_postprocess=False)
    return outputs

def merge_detections(all_boxes, all_scores, all_classes, shape_hw, num_classes=3, 
                     box_nms_thresh=0.99, max_detection_per_im=10000):
    # select from the union of all results
    num_boxes = len(all_boxes)
    
    # +1 because fast_rcnn_inference expects background scores as well
    all_scores_2d = torch.zeros(num_boxes, num_classes + 1, device=all_boxes.device)
    for idx, cls, score in zip(count(), all_classes, all_scores):
        all_scores_2d[idx, cls] = score

    merged_instances, _ = fast_rcnn_inference_single_image(
        all_boxes,
        all_scores_2d,
        shape_hw,
        1e-8,
        box_nms_thresh,
        max_detection_per_im,
    )
    
    return merged_instances

def rescale_detected_boxes(augmented_inputs, merged_instances, tfms):
    augmented_instances = []
    for input, tfm in zip(augmented_inputs, tfms):
        # Transform the target box to the augmented image's coordinate space
        pred_boxes = merged_instances.pred_boxes.tensor.cpu().numpy()
        pred_boxes = torch.from_numpy(tfm.apply_box(pred_boxes))

        aug_instances = Instances(
            image_size=input["image"].shape[1:3],
            pred_boxes=Boxes(pred_boxes),
            pred_classes=merged_instances.pred_classes,
            scores=merged_instances.scores,
        )
        augmented_instances.append(aug_instances)
    return augmented_instances

def reduce_pred_masks(outputs, tfms):
    # Should apply inverse transforms on masks.
    # We assume only resize & flip are used. pred_masks is a scale-invariant
    # representation, so we handle flip specially
    for output, tfm in zip(outputs, tfms):
        if any(isinstance(t, HFlipTransform) for t in tfm.transforms):
            output.pred_masks = output.pred_masks.flip(dims=[3])
    all_pred_masks = torch.stack([o.pred_masks for o in outputs], dim=0)
    avg_pred_masks = torch.mean(all_pred_masks, dim=0)
    return avg_pred_masks

def ensemble(inp_dict, list_configs, list_predictors, 
            mask_nms_thresh=0.1, max_detection_per_im=10000,
            conf_thresh=[0.3,0.5,0.6]):
    # read image
    inp = copy.copy(inp_dict)
    image = read_image(inp.pop("file_name"), list_predictors[0].model.input_format)
    image = torch.from_numpy(np.ascontiguousarray(image.transpose(2, 0, 1)))  # CHW
    inp["image"] = image
    if "height" not in inp and "width" not in inp:
        inp["height"] = image.shape[1]
        inp["width"] = image.shape[2]
    orig_shape = (inp["height"], inp["width"])
    
    # dataset mapper for each model
    list_tta_mappers = []
    list_augmented_inputs = []
    list_tfms = []
    
    all_boxes = []
    all_scores = []
    all_classes = []
    
    for cfg, predictor in zip(list_configs, list_predictors):
        tta_mapper = DatasetMapperTTA(cfg)
        list_tta_mappers.append(tta_mapper)
        augmented_inputs = tta_mapper(inp)
        
        tfms = [x.pop("transforms") for x in augmented_inputs]
        list_augmented_inputs.append(augmented_inputs)
        list_tfms.append(tfms)
        
#         return augmented_inputs
        with turn_off_roi_heads(predictor.model, ['mask_on', 'keypoint_on']), torch.no_grad():
            outputs = model_batch_forward(predictor.model, augmented_inputs)
            
        for output, tfm in zip(outputs, tfms):
            # Need to inverse the transforms on boxes, to obtain results on original image
            pred_boxes = output.pred_boxes.tensor
            original_pred_boxes = tfm.inverse().apply_box(pred_boxes.cpu().numpy())
            all_boxes.append(torch.from_numpy(original_pred_boxes).to(pred_boxes.device))

            all_scores.extend(output.scores)
            all_classes.extend(output.pred_classes)
            
    all_boxes = torch.cat(all_boxes, dim=0)
    
    merged_instances = merge_detections(all_boxes, all_scores, all_classes, orig_shape)

    # filter by confidence score
    pred_class = torch.mode(merged_instances.pred_classes)[0]
    take = merged_instances.scores >= conf_thresh[pred_class]
    merged_instances = merged_instances[take]
    
    list_outputs = []
    for cfg, predictor, augmented_inputs, tfms in zip(list_configs, list_predictors, list_augmented_inputs, list_tfms):
        augmented_instances = rescale_detected_boxes(
            augmented_inputs, merged_instances, tfms
        )
                  
        with torch.no_grad():
            outputs = model_batch_forward(predictor.model, augmented_inputs, augmented_instances)
        
        del augmented_inputs, augmented_instances
        
        list_outputs.extend(outputs)
        
    list_tfms = itertools.chain.from_iterable(list_tfms)
    
    merged_instances.pred_masks = reduce_pred_masks(list_outputs, list_tfms)
    merged_instances = detector_postprocess(merged_instances, *orig_shape)
    
    final_outputs = {'instances':merged_instances}
    final_outputs = custom_nms(final_outputs, nms_thresh=mask_nms_thresh)
    # final_outputs = post_process_output(final_outputs, mask_nms_thresh=mask_nms_thresh, conf_thresh=conf_thresh)
    
    return final_outputs

In [ ]:
cfg1, predictor1 = get_predictor_v21()
cfg2, predictor2 = get_predictor_v16()

In [ ]:
X_cols = pd.read_csv('../input/sartorius-stage2-models/features_ver_5_ens_m1022_m1018.csv')['features'].tolist()

y_col = 'FP'

stage2_thresh = 0.3

import pickle
with open('../input/sartorius-stage2-models/model_ver_5_ens_m1022_m1018.pkl', 'rb') as f:
    stage2_model = pickle.load(f)

In [ ]:
ids, masks=[],[]
test_names = (dataDir/'test').ls()

In [ ]:
FINAL_THRESH = [0.5, 0.8, 0.8]

In [ ]:
for fn in tqdm(test_names, total=len(test_names)):
    d = {'file_name':str(fn), 'height':520, 'width':704}
    im = cv2.imread(d['file_name'])
    pred = ensemble(d, 
                   [cfg1, cfg2], 
                   [predictor1, predictor2],
                  conf_thresh=FINAL_THRESH)
    
    pred['instances'] = pred['instances'].to('cpu')
    
    pred_class = torch.mode(pred['instances'].pred_classes)[0]
    
    # stage 2 prediction
    features = get_features(im, pred)
    
    features['instance_num'] = np.arange(0, len(features))
    features['cell_type'] = [pred_class]*len(features)

    features[X_cols] = features[X_cols].fillna(-1)
    
    predicted_iou = stage2_model.predict(features[X_cols])
    features['pred_FP'] = predicted_iou < stage2_thresh
    TP_list = features[features['pred_FP']==0].instance_num.tolist()
    pred['instances'] = pred['instances'][TP_list]
    pred['instances'].pred_masks = (pred['instances'].pred_masks > 0.5)
    
    encoded_masks = get_masks(pred)
    for enc in encoded_masks:
        ids.append(fn.stem)
        masks.append(enc)

In [ ]:
final_df = pd.DataFrame({'id':ids, 'predicted':masks})
final_df.to_csv('submission.csv', index=False)
final_df.head()

In [ ]:
ids = final_df.id.unique()

In [ ]:
# visualize sample
i = 0
fname = test_names[i]
encoded_masks = final_df[final_df.id == ids[i]]['predicted']
print(len(encoded_masks))

_, axs = plt.subplots(1,2, figsize=(40,15))
axs[1].imshow(cv2.imread(str(fname)))
for enc in encoded_masks:
    dec = rle_decode(enc)
    axs[0].imshow(np.ma.masked_where(dec==0, dec))

In [ ]:
# visualize sample
i = 1
fname = test_names[i]
encoded_masks = final_df[final_df.id == ids[i]]['predicted']
print(len(encoded_masks))

_, axs = plt.subplots(1,2, figsize=(40,15))
axs[1].imshow(cv2.imread(str(fname)))
for enc in encoded_masks:
    dec = rle_decode(enc)
    axs[0].imshow(np.ma.masked_where(dec==0, dec))

In [ ]:
# visualize sample
i = 2
fname = test_names[i]
encoded_masks = final_df[final_df.id == ids[i]]['predicted']
print(len(encoded_masks))

_, axs = plt.subplots(1,2, figsize=(40,15))
axs[1].imshow(cv2.imread(str(fname)))
for enc in encoded_masks:
    dec = rle_decode(enc)
    axs[0].imshow(np.ma.masked_where(dec==0, dec))

In [ ]:
ls